## Análise pós-clusterização
***

In [156]:
import pandas as pd
import numpy as np

In [249]:
path = '/Users/felipebarreto/Desktop/TCC/Bases geradas/'
df1 = pd.read_csv('/Users/felipebarreto/Desktop/dados_acidentes_tratados_completo.csv', sep = ',')   ## dados originais tratados
df2 = pd.read_csv(path + 'centroides_padronizados.csv', sep = ';')   ## centróides de cada cluster     //
df3 = pd.read_csv(path + 'variaveis_quanti_agg.csv', sep = ';')      ## igual ao df4, sem padronização // 
df4 = pd.read_csv(path + 'variaveis_quanti_pad_agg.csv', sep = ';')  ## igual ao df3, com padronização // 
df5 = pd.read_csv(path + 'medias_var_cluster.csv', sep = ';', decimal= '.')        ## médias de cada cluster         // 
df6 = pd.read_csv(path + 'clusters_7.csv', sep = ';') # var quantitativas completas, nem todas foram utilizadas para a clusterização

In [343]:
def _status_qtd(var):
      """
      Função para cálculo de frequência e porcentagem relativa a cada grupo

      Parâmetro:
      ----------
      var - variável a ser sumarizada e normalizada
      """
      a = pd.merge(df1, df6[['municipio','cluster']], how='left')\
            .groupby(['cluster',var])[['id']]\
            .count()\
            .reset_index()\
            .rename(columns={var:'status','id':var})\
            .set_index(['cluster','status'])
      a_pct = a.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))\
            .rename(columns={var:var+'_pct'})
      df_completo = a.join(a_pct)
      return df_completo

In [378]:
## Crisndo um dataframe com o share das variáveis categóricas
df_share = _status_qtd('tracado_via_tipo_1')\
    .join(_status_qtd('is_night'))\
    .join(_status_qtd('is_weekend'))\
    .join(_status_qtd('is_single_lane'))\
    .reset_index()\
    .query("status == True")

In [379]:
display(
    df2.style.background_gradient(cmap='Greens').set_precision(2),
    df5.style.background_gradient(cmap='Reds').set_precision(2),
    df_share.style.background_gradient(cmap='Blues').set_precision(2)
    )

,Variavel,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Cluster 5
0,mortos,5.11,-0.22,-0.16,-0.32,1.84
1,feridos_leves,6.54,-0.28,-0.02,-0.26,1.70
2,feridos_graves,6.46,-0.26,-0.12,-0.27,1.74
3,ilesos,7.03,-0.26,-0.01,-0.26,1.51
4,freq,6.90,-0.28,0.01,-0.26,1.62
5,share_tracado_via_tipo_1,0.42,0.50,0.15,-1.37,0.02
6,share_is_night,-0.25,0.10,-0.14,-0.11,-0.13
7,share_is_weekend,-0.23,0.04,-0.23,0.06,-0.05
8,share_is_single_lane,-1.83,0.45,-2.11,0.41,-0.78


,cluster,mortos,feridos_leves,feridos_graves,ilesos,freq,share_tracado_via_tipo_1,share_is_night,share_is_weekend,share_is_single_lane
0,Cluster 1,330.33,5434.96,1752.12,20100.00,13029.42,0.80,0.36,0.44,0.24
1,Cluster 2,27.04,154.29,66.81,352.11,271.88,0.82,0.43,0.49,0.96
2,Cluster 3,30.57,351.57,101.76,1023.44,791.34,0.73,0.38,0.44,0.15
3,Cluster 4,21.64,170.06,63.53,358.45,302.33,0.36,0.39,0.49,0.95
4,Cluster 5,144.31,1689.63,568.29,5151.79,3644.69,0.70,0.38,0.47,0.57


,cluster,status,tracado_via_tipo_1,tracado_via_tipo_1_pct,is_night,is_night_pct,is_weekend,is_weekend_pct,is_single_lane,is_single_lane_pct
1,1,True,249718,79.86,112859,36.09,136865,43.77,65250,20.87
3,2,True,263650,76.72,142797,41.55,168098,48.91,323360,94.09
5,3,True,153379,70.23,86573,39.64,103450,47.37,32632,14.94
7,4,True,66154,43.41,56530,37.10,73951,48.53,145305,95.36
9,5,True,496002,69.79,269882,37.97,331268,46.61,342236,48.15


### Análise

In [405]:
df = df3.drop(['pessoas','ignorados','feridos','veiculos'], axis=1)[[
    'municipio','cluster','freq','mortos','feridos_graves','feridos_leves','ilesos','share_tracado_via_tipo_1',
    'share_tracado_via_tipo_2','share_is_night','share_is_weekend','share_is_single_lane'
]].drop_duplicates(subset = ['municipio'])


## Separação dos clusters (c1,c2,c3,c4,c5,c6,c7)
n = df.cluster.nunique()
for i in range(1,n + 1):
    globals()['c' + str(i)] = df[df.cluster == i].drop(['cluster'], axis=1)

df.head(5)

,municipio,cluster,freq,mortos,feridos_graves,feridos_leves,ilesos,share_tracado_via_tipo_1,share_tracado_via_tipo_2,share_is_night,share_is_weekend,share_is_single_lane
0,curitiba,1,23714,581,2872,10382,36629,0.834781,0.155604,0.375221,0.438306,0.106477
1,sao jose,1,20102,191,1928,8828,31326,0.784648,0.205552,0.369018,0.414785,0.043379
2,guarulhos,1,18109,430,1040,8282,27409,0.942239,0.052018,0.410293,0.422055,0.055221
3,betim,1,17601,329,1718,6840,29551,0.784103,0.208625,0.361911,0.489234,0.048804
4,duque de caxias,1,17531,407,1273,6388,28066,0.842393,0.153785,0.380811,0.430152,0.195710


In [160]:
def _loc(df, cluster, col):
    """
    Função para buscar um valor específico na tabela, dada uma variável

    Parâmetros:
    -----------
    df ------> dataframe 
    cluster -> cluster escolhido
    col -----> variável a ser buscada
    """
    return(round(df[df.cluster == 'Cluster '+str(cluster)][col].iloc[0],2))

In [401]:
## Tamanho dos clusters // Quantidade de municípios
print(
    f""">> DADOS GERAIS - Acidentes de 2007 a 2021 (16 anos)<<
    - Total de municípios analisados: {len(df)}
    - Mortos = {df.mortos.sum()} | Média = {round(df.mortos.mean(),2)}
    - Feridos graves = {df.feridos_graves.sum()} | Média = {round(df.feridos_graves.mean(),2)}
    - Feridos leves = {df.feridos_leves.sum()} | Média = {round(df.feridos_leves.mean(),2)}
    - Ilesos = {df.ilesos.sum()} | Média = {round(df.ilesos.mean(),2)}
    - Share de tração via tipo 1 = {round(100*df1.tracado_via_tipo_1.value_counts(normalize=True)[1],2)}%
    - Share de é noite = {round(100*df1.is_night.value_counts(normalize=True)[1],2)}%
    - Share de é final de semana = {round(100*df1.is_weekend.value_counts(normalize=True)[1],2)}%
    - Share de é pista simples = {round(100*df1.is_single_lane.value_counts(normalize=True)[1],2)}%
    """
)

for i in range(1,n+1):

    print(f"""Cluster {i}:
        - {len(globals()['c' + str(i)])} municípios --> {round(100*df[df.cluster == i].shape[0] / df.shape[0],2)}% do total
        - Mortos = {globals()['c' + str(i)].mortos.sum()},  {round(100*globals()['c' + str(i)].mortos.sum() / df.mortos.sum(),2)}% do total | Média = {_loc(df5, i, col='mortos')}, {round(100*_loc(df5, i, col='mortos') / df.mortos.mean(),2)}% da média geral
        - Feridos graves = {globals()['c' + str(i)].feridos_graves.sum()},  {round(100*globals()['c' + str(i)].feridos_graves.sum() / df.feridos_graves.sum(),2)}% do total | Média = {_loc(df5, i, col='feridos_graves')}, {round(100*_loc(df5, i, col='feridos_graves') / df.feridos_graves.mean(),2)}% da média geral
        - Feridos Leves = {globals()['c' + str(i)].feridos_leves.sum()},  {round(100*globals()['c' + str(i)].feridos_leves.sum() / df.feridos_leves.sum(),2)}% do total | Média = {_loc(df5, i, col='feridos_leves')}, {round(100*_loc(df5, i, col='feridos_leves') / df.feridos_leves.mean(),2)}% da média geral
        - Ilesos = {globals()['c' + str(i)].ilesos.sum()},  {round(100*globals()['c' + str(i)].ilesos.sum() / df.ilesos.sum(),2)}% do total | Média = {_loc(df5, i, col='ilesos')}, {round(100*_loc(df5, i, col='ilesos') / df.ilesos.mean(),2)}% da média geral
        - Média do share de traçado via tipo 1 = {round(100*_loc(df5, i, col='share_tracado_via_tipo_1'),2)}%
        - Média do share de é noite = {round(100*_loc(df5, i, col='share_is_night'),2)}%
        - Média do share de é final de semana = {round(100*_loc(df5, i, col='share_is_weekend'),2)}%
        - Média do share de é pista simples = {round(100*_loc(df5, i, col='share_is_single_lane'),2)}%
    """)

>> DADOS GERAIS - Acidentes de 2007 a 2021 (16 anos)<<
    - Total de municípios analisados: 2263
    - Mortos = 89587 | Média = 39.59
    - Feridos graves = 297414 | Média = 131.42
    - Feridos leves = 837677 | Média = 370.16
    - Ilesos = 2395201 | Média = 1058.42
    - Share de tração via tipo 1 = 70.71%
    - Share de é noite = 38.47%
    - Share de é final de semana = 46.82%
    - Share de é pista simples = 52.29%
    
Cluster 1:
        - 24 municípios --> 1.06% do total
        - Mortos = 7928,  8.85% do total | Média = 330.33, 834.43% da média geral
        - Feridos graves = 42051,  14.14% do total | Média = 1752.12, 1333.17% da média geral
        - Feridos Leves = 130439,  15.57% do total | Média = 5434.96, 1468.26% da média geral
        - Ilesos = 482400,  20.14% do total | Média = 20100.0, 1899.06% da média geral
        - Média do share de traçado via tipo 1 = 80.0%
        - Média do share de é noite = 36.0%
        - Média do share de é final de semana = 44.0%
      

#### Observações:

- Total de Municípios analisados: 2263
- Municípios por região:
    - N ---> 
    - NE --> 
    - CO --> 
    - S ---> 
    - SE -->  
- Municípios por UF:
    - sdc

## Cluster 1

In [402]:
c1

,municipio,freq,mortos,feridos_graves,feridos_leves,ilesos,share_tracado_via_tipo_1,share_tracado_via_tipo_2,share_is_night,share_is_weekend,share_is_single_lane
0,curitiba,23714,581,2872,10382,36629,0.834781,0.155604,0.375221,0.438306,0.106477
1,sao jose,20102,191,1928,8828,31326,0.784648,0.205552,0.369018,0.414785,0.043379
2,guarulhos,18109,430,1040,8282,27409,0.942239,0.052018,0.410293,0.422055,0.055221
3,betim,17601,329,1718,6840,29551,0.784103,0.208625,0.361911,0.489234,0.048804
4,duque de caxias,17531,407,1273,6388,28066,0.842393,0.153785,0.380811,0.430152,0.195710
5,serra,17338,386,3124,5287,27859,0.632138,0.362729,0.313185,0.412331,0.207060
6,brasilia,14992,621,2646,10589,20608,0.875800,0.105990,0.394144,0.494731,0.125000
7,cariacica,13508,182,2257,4060,20983,0.694181,0.299822,0.305597,0.391694,0.143841
8,porto velho,13101,378,2165,7503,20390,0.669109,0.324708,0.371956,0.429357,0.233952
9,palhoca,13075,229,1926,6486,18441,0.779273,0.208184,0.403365,0.490554,0.372696
